In [1]:
%conda install conda=24.1.2

Retrieving notices: ...working... done
done
Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::bleach==6.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::importlib_metadata==7.0.1=hd8ed1ab_0
  - conda-forge/noarch::jupyter_client==8.6.0=pyhd8ed1ab_0
  - conda-forge/noarch::s3transfer==0.10.0=pyhd8ed1ab_0
  - conda-forge/noarch::ipykernel==6.29.0=pyhd33586a_0
  - https://aws-ml-conda-ec2.s3.us-west-2.amazonaws.com/linux-64::torchdata==0.7.0=py310
  - conda-forge/noarch::nbclient==0.8.0=pyhd8ed1ab_0
  - conda-forge/linux-64::onnx==1.13.1=py310ha3deec4_2
  - conda-forge/noarch::bokeh==3.3.4=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.8.2=py310h62c0568_0
  - conda-forge/noarch::nbconvert-core==7.14.2=pyhd8ed1ab_0
  - conda-forge/linux-64::statsmodels==0.14.1=py310h1f7b6fc_0
  - conda-forge/noarch::jupyter_server==2.12.5=pyhd8ed1ab_0
  - conda-forge/noarch::seab

In [1]:
%conda install transformers accelerate

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/pytorch_p310

  added / updated specs:
    - accelerate
    - transformers


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    accelerate-0.27.0          |     pyhd8ed1ab_0         184 KB  conda-forge
    aiohttp-3.9.3              |  py310h2372a71_0         677 KB  conda-forge
    aiosignal-1.3.1            |     pyhd8ed1ab_0          12 KB  conda-forge
    async-timeout-4.0.3        |     pyhd8ed1ab_0          11 KB  conda-forge
    aws-c-auth-0.7.3           |       h28f7589_1          99 KB  conda-forge
    aws-c-cal-0.6.1            |       hc309b26_1          50 KB  conda-forge
    aws-c-common-0.9.0         |       hd590300_0         193 KB  conda-forge
    aws-c-compression-0.2.17   |       h4d4d85c_2          19 KB  c

In [1]:
import os
import json
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    MistralForCausalLM,
)
from datasets import load_dataset, load_metric

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sagemaker
import boto3
import tarfile
import random

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::940119374655:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker bucket: sagemaker-us-east-1-940119374655
sagemaker session region: us-east-1


In [2]:
student_id = "mistralai/Mistral-7B-v0.1"
teacher_id = "mistralai/Mixtral-8x7B-v0.1"
dataset_id = "cais/mmlu"
dataset_config = "all"
s3_prefix_dataset = "knowledge_distill_mistral"

In [4]:
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_id)
student_tokenizer = AutoTokenizer.from_pretrained(student_id)

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
sample = "Hopefully these two values are the same"

assert teacher_tokenizer(sample) == student_tokenizer(sample), (
    "Tokenizers are the same "
    f"{teacher_tokenizer(sample)} != {student_tokenizer(sample)}"
)

In [6]:
del teacher_tokenizer
del student_tokenizer

In [7]:
idx_mapping = {
    1: "A",
    2: "B",
    3: "C",
    4: "D"
}

In [8]:
def hf_dataset_to_dict(dataset):
    return {key: dataset[key] for key in dataset.column_names}

In [9]:
def random_few_shot():
    random_key = random.choice(list(few_shot_dict.keys()))
    random_value = few_shot_dict[random_key]

    return random_value

In [10]:
def append_question_to_prompt(data, show_answer=False):
    prompt = ''
    prompt += f"{data['question']}\n\n"

    for idx, choice in enumerate(data['choices'], start=1):
        prompt += f"{idx_mapping[idx]}. {choice}\n"
    
    if show_answer:
        prompt += f"\nAnswer: {data['answer']}\n\n"
    else:
        prompt += f"\nAnswer:"
        
    return prompt

In [11]:
def get_subject_few_shot(subject_filter):
    index_list = [i for i, subject in enumerate(dev_dataset['subject']) if subject == subject_filter]

    new_data = [
        {
            'question': dev_dataset['question'][i],
            'subject': dev_dataset['subject'][i],
            'choices': dev_dataset['choices'][i],
            'answer': dev_dataset['answer'][i]
        }
        for i in index_list
    ]
    
    few_shot_prompt = ''
    
    for data in new_data:
        few_shot_prompt += append_question_to_prompt(data, show_answer=True)
        

    return few_shot_prompt

In [12]:
def format_subject(subject):
    l = subject.split("_")
    s = ""
    for entry in l:
        s += " " + entry
    return s

In [13]:
def test_process(data):
    subject = data['subject']

    prompt = f"The following are multiple choice questions (with answers) about {format_subject(subject)}.\n\n"
    prompt += few_shot_dict[subject]
    prompt += append_question_to_prompt(data)
    
    tokenized_prompt = tokenizer(prompt, truncation=True, max_length=4096, padding=True)

    return tokenized_prompt

In [14]:
def train_process(data):
    prompt = f"The following are multiple choice questions (with answers) about a random subject.\n\n"
    prompt += random_few_shot()
    prompt += append_question_to_prompt(data)
    
    tokenized_prompt = tokenizer(prompt, truncation=True, max_length=4096, padding=True)

    return tokenized_prompt

In [15]:
dataset = load_dataset(dataset_id, dataset_config)

train_dataset = dataset['auxiliary_train']
test_dataset = dataset['test']
dev_dataset = dataset['dev']

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [16]:
few_shot_dict = {}
dev_dict = hf_dataset_to_dict(dev_dataset)
subjects = list(set(dev_dict['subject']))
for subject in subjects:
    few_shot_dict[subject] = get_subject_few_shot(subject)

In [18]:
tokenizer = AutoTokenizer.from_pretrained(teacher_id)
tokenizer.pad_token = tokenizer.eos_token

In [20]:
train_dataset = train_dataset.map(train_process)

Map:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [21]:
test_dataset = test_dataset.map(test_process)

Map:   0%|          | 0/14042 [00:00<?, ? examples/s]

In [22]:
train_dataset = train_dataset.remove_columns(['question', 'subject', 'choices'])
print(train_dataset.features)

{'answer': ClassLabel(names=['A', 'B', 'C', 'D'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [23]:
test_dataset = test_dataset.remove_columns(['question', 'subject', 'choices'])
print(test_dataset.features)

{'answer': ClassLabel(names=['A', 'B', 'C', 'D'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [24]:
train_dataset.set_format("torch")
test_dataset.set_format("torch")

In [25]:
dataset_path = f's3://{sess.default_bucket()}/{s3_prefix_dataset}'
train_dataset_path = dataset_path + '/dataset/train'
test_dataset_path = dataset_path + '/dataset/test'

train_dataset.save_to_disk(train_dataset_path)
test_dataset.save_to_disk(test_dataset_path)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Saving the dataset (0/1 shards):   0%|          | 0/99842 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14042 [00:00<?, ? examples/s]

In [17]:
# TO sTART NEW RUN HERE TO BOTTOM
import sagemaker
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
name = 'mistral-mmlu'
dataset_path = f's3://{sess.default_bucket()}/{s3_prefix_dataset}'
train_dataset_path = dataset_path + '/dataset/train'
test_dataset_path = dataset_path + '/dataset/test'

In [38]:
hyperparameters={
        'teacher_model_id': teacher_id,
        'student_model_id': student_id,
        'num_epochs': 1,
        'lr': 6e-5,
        'fp16': True,
        'temperature': 4.0,
        'alpha': 0.5,
        'save_strategy': 'epoch',
        'evaluation_strategy': 'epoch',
}

In [39]:
import time
job_name = f'{name}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
print(job_name)

mistral-mmlu-2024-03-11-01-57-14


In [40]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',
    source_dir           = 'scripts',
    instance_type        = 'ml.p4d.24xlarge',
    instance_count       = 1,
    base_job_name        = job_name,
    role                 = role,
    disable_output_compression = False,
    input_mode           = 'File',
    transformers_version = '4.36',
    pytorch_version      = '2.1',
    py_version           = 'py310',
    hyperparameters = hyperparameters
)

In [41]:
print(dataset_path)

s3://sagemaker-us-east-1-940119374655/knowledge_distill_mistral


In [42]:
huggingface_estimator.fit({
        'train_data': train_dataset_path,
        'test_data': test_dataset_path
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: mistral-mmlu-2024-03-11-01-57-14-2024-03-11-01-57-36-978


2024-03-11 01:57:37 Starting - Starting the training job...
2024-03-11 01:57:43 Pending - Training job waiting for capacity...
2024-03-11 01:58:19 Pending - Preparing the instances for training........................
2024-03-11 02:02:26 Downloading - Downloading input data......
2024-03-11 02:03:36 Downloading - Downloading the training image............
2024-03-11 02:05:11 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-03-11 02:06:27,395 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-03-11 02:06:27,490 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-03-11 02:06:27,499 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-03-11 02:06:27,501 sagemaker_pytorch_container.training INFO     Invoking user training scr

UnexpectedStatusException: Error for Training job mistral-mmlu-2024-03-11-01-57-14-2024-03-11-01-57-36-978: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacty of 39.39 GiB of which 206.31 MiB is free. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 38.78 GiB is allocated by PyTorch, and 601.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF"
Command "/opt/conda/bin/python3.10 train.py --alpha 0.5 --evaluation_strategy epoch --fp16 True --lr 6e-05 --num_epochs 1 --save_strategy epoch --student_model_id mistralai/Mistral-7B-v0.1 --teacher_model_id mistralai/Mixtral-8x7B-v0.1 --temperature 4.0", exit code: 1